<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第4章：从零实现 GPT 模型以生成文本


In [1]:
# 导入 importlib.metadata 中的 version 函数，用于获取已安装库的版本号
from importlib.metadata import version

# -------------------------------------------------------
# 输出 matplotlib 库的版本号
# version("matplotlib") 会返回当前安装的 matplotlib 版本字符串，例如 "3.8.0"
# -------------------------------------------------------
print("matplotlib version:", version("matplotlib"))

# -------------------------------------------------------
# 输出 torch 库的版本号
# version("torch") 会返回当前安装的 PyTorch 版本字符串，例如 "2.2.0"
# -------------------------------------------------------
print("torch version:", version("torch"))

# -------------------------------------------------------
# 输出 tiktoken 库的版本号
# version("tiktoken") 会返回当前安装的 tiktoken 版本字符串，例如 "0.7.0"
# -------------------------------------------------------
print("tiktoken version:", version("tiktoken"))


matplotlib version: 3.10.0
torch version: 2.9.0+cpu
tiktoken version: 0.12.0


- 本章中，我们将实现一个类似 GPT 的大型语言模型（LLM）架构；下一章将重点介绍训练该 LLM


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/01.webp" width="500px">

## 4.1 编写大型语言模型（LLM）架构代码


- 第 1 章中我们讨论了像 GPT 和 Llama 这样的模型，它们是基于原始 Transformer 架构中的解码器部分，并通过**逐词生成**的方式输出文本。
- 因此，这类大型语言模型（LLMs）通常被称为“**解码器型（decoder-like）LLMs**”。
- 与传统的深度学习模型相比，LLM 的体量更大，这主要是因为它们拥有**海量的参数数量**，而不是因为代码量庞大。
- 我们将看到，在 LLM 的架构中，许多模块和结构都是**重复出现**的。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/02.webp" width="400px">

- 在前几章中，我们为了方便展示，使用了**较小的词嵌入维度**来表示输入和输出，使得计算过程可以清晰地呈现在一页纸上。
- 而在本章中，我们将采用与 **GPT-2 小型模型（small GPT-2）** 相近的嵌入与模型规模。
- 我们将特别实现 **最小版 GPT-2 模型（约 1.24 亿参数）** 的架构，参考自 Radford 等人的论文  
  [《Language Models are Unsupervised Multitask Learners》](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)。  
  （注意：论文最初报告参数量为 1.17 亿，但在模型权重仓库中后来修正为 1.24 亿。）
- 第 6 章将展示如何在我们的实现中**加载预训练权重**，并且该实现也兼容更大规模的 GPT-2 模型（3.45 亿、7.62 亿、15.42 亿参数版本）。


- 具有 1.24 亿参数的 GPT-2 模型的配置细节如下：


In [2]:
# ============================================================
# GPT-124M 模型配置字典
# ============================================================
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # 词表大小 (Vocabulary size)
                             # - 表示模型可识别的不同 token 的总数
                             # - 包括字母、符号、特殊 token 等

    "context_length": 1024,  # 上下文长度 (Context length)
                             # - 模型一次可以处理的最大 token 数
                             # - 在自回归生成中，超过该长度的 token 会被截断或滑动窗口处理

    "emb_dim": 768,          # 嵌入维度 (Embedding dimension)
                             # - 输入 token embedding 和隐藏状态的维度
                             # - 决定每个 token 表示的向量大小

    "n_heads": 12,           # 注意力头数量 (Number of attention heads)
                             # - 多头注意力中并行头的数量
                             # - 每个头独立学习不同的注意力模式

    "n_layers": 12,          # Transformer 层数 (Number of layers)
                             # - 堆叠的自注意力 + 前馈网络层数
                             # - 决定模型的深度

    "drop_rate": 0.1,        # Dropout 比例 (Dropout rate)
                             # - 在训练时随机置零的比例，用于防止过拟合

    "qkv_bias": False        # Q/K/V 偏置 (Query-Key-Value bias)
                             # - 是否在 Q/K/V 线性映射中使用偏置项
}


- 我们使用较短的变量名，以避免后续代码行过长。  
- `"vocab_size"` 表示词汇表大小为 50,257 个词，对应第 2 章中介绍的 BPE 分词器。  
- `"context_length"` 表示模型能够处理的最大输入 token 数量，对应第 2 章中讲到的位置嵌入机制。  
- `"emb_dim"` 是输入 token 的嵌入维度，将每个输入 token 转换为一个 768 维向量。  
- `"n_heads"` 是多头注意力机制中的注意力头数量，这在第 3 章中已经实现。  
- `"n_layers"` 是模型中 Transformer 块的层数，我们将在后续小节中实现。  
- `"drop_rate"` 是 dropout 机制的强度，在第 3 章中讨论过；值为 0.1 表示在训练过程中随机丢弃 10% 的隐藏单元，以防止过拟合。  
- `"qkv_bias"` 决定多头注意力机制（第 3 章中的实现）中的 `Linear` 层在计算 Query（Q）、Key（K）和 Value（V）张量时是否包含偏置向量；我们将在此禁用该选项，这是现代 LLM 的常见做法。不过，当我们在第 5 章加载 OpenAI 发布的 GPT-2 预训练权重时，将会重新讨论这一点。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/03.webp" width="500px">

In [3]:
# ============================================================
# Dummy GPT 模型实现
# ============================================================

import torch
import torch.nn as nn

# ================================
# 主模型类：DummyGPTModel
# ================================
class DummyGPTModel(nn.Module):
    """
    一个简化的 GPT 模型，用于教学或调试。
    模型结构包含：
      1. token embedding（将 token ID 转为向量）
      2. position embedding（位置编码）
      3. dropout（防止过拟合）
      4. 若干占位 Transformer blocks（这里不执行实际计算）
      5. 最终的 LayerNorm（占位）
      6. 输出线性层（投影到词表大小，生成 logits）
    """

    def __init__(self, cfg):
        """
        初始化模型
        参数:
            cfg: 配置字典，包含模型参数，例如：
                - vocab_size: 词表大小
                - context_length: 最大上下文长度
                - emb_dim: token 嵌入维度
                - n_layers: Transformer 层数
                - drop_rate: dropout 比例
        """
        super().__init__()

        # =============================
        # Token Embedding 层
        # 输入: [batch_size, seq_len] 的 token ID
        # 输出: [batch_size, seq_len, emb_dim]
        # 功能: 将离散 token 编码为连续向量
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])

        # =============================
        # Position Embedding 层
        # 输入: [seq_len] 的位置索引 0..seq_len-1
        # 输出: [seq_len, emb_dim]
        # 功能: 提供序列中每个 token 的位置信息
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])

        # =============================
        # Dropout 层
        # 功能: 在训练时随机丢弃部分神经元，防止过拟合
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # =============================
        # Transformer Block 堆叠（占位）
        # nn.Sequential 可以将多个模块按顺序组合
        # 这里使用 DummyTransformerBlock 占位，返回输入
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        # =============================
        # 最终 LayerNorm（占位）
        # 功能: 模仿 LayerNorm 接口，但不做实际归一化
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])

        # =============================
        # 输出头
        # Linear 将最后的嵌入投影到词表大小，用于计算每个 token 的 logits
        # 输出: [batch_size, seq_len, vocab_size]
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        """
        前向传播函数
        参数:
            in_idx: [batch_size, seq_len] 的 token 索引张量
        返回:
            logits: [batch_size, seq_len, vocab_size] 的预测分布
        """
        batch_size, seq_len = in_idx.shape

        # =============================
        # Token embedding
        # 输入: token ID
        # 输出: [batch_size, seq_len, emb_dim]
        tok_embeds = self.tok_emb(in_idx)

        # =============================
        # Position embedding
        # torch.arange(seq_len) 生成位置索引 0..seq_len-1
        # 输出: [seq_len, emb_dim]
        # 通过 broadcasting 可与 tok_embeds 相加
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        # =============================
        # 合并 token embedding 与 position embedding
        # 输出: [batch_size, seq_len, emb_dim]
        x = tok_embeds + pos_embeds

        # =============================
        # Dropout
        # 随机丢弃部分元素，训练时可防止过拟合
        x = self.drop_emb(x)

        # =============================
        # Transformer blocks
        # 这里为占位，DummyTransformerBlock 直接返回输入
        x = self.trf_blocks(x)

        # =============================
        # LayerNorm
        # 占位 LayerNorm，不做实际归一化
        x = self.final_norm(x)

        # =============================
        # 输出头，将 embedding 投影到词表大小
        # 输出: [batch_size, seq_len, vocab_size]
        logits = self.out_head(x)

        return logits


# ================================
# Dummy Transformer Block 占位类
# ================================
class DummyTransformerBlock(nn.Module):
    """
    占位 Transformer Block
    功能:
        - 不进行实际计算
        - 模仿接口，方便教学或模型堆叠
    """

    def __init__(self, cfg):
        super().__init__()
        # 真实模型中会有:
        # - Multi-Head Self-Attention
        # - Feed-Forward Network
        # - LayerNorm
        # 这里占位留空

    def forward(self, x):
        # 直接返回输入
        return x


# ================================
# Dummy LayerNorm 占位类
# ================================
class DummyLayerNorm(nn.Module):
    """
    占位 LayerNorm
    功能:
        - 不进行实际归一化
        - 保持接口一致
    """

    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 真实 LayerNorm 会初始化权重和偏置
        # 这里留空

    def forward(self, x):
        # 直接返回输入
        return x


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/04.webp?123" width="500px">

In [4]:
# ============================================================
# 使用 tiktoken 对文本进行编码，并形成一个 batch
# ============================================================

import tiktoken

# =============================
# 1️ 获取 GPT-2 tokenizer
# tiktoken 提供了高效的 BPE 编码器
# "gpt2" 对应 GPT-2 词表和 BPE 规则
tokenizer = tiktoken.get_encoding("gpt2")

# =============================
# 2️ 准备一个空列表用于存储 batch 中的 token 张量
batch = []

# =============================
# 3️ 定义两段示例文本
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

# =============================
# 4️ 对每段文本进行编码，将其转换为整数 token ID
# tokenizer.encode 返回一个 list[int]，每个元素是词表中的索引
# torch.tensor 将 list[int] 转为 PyTorch 张量
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

# =============================
# 5️ 将两个序列堆叠为一个 batch 张量
# torch.stack 会在第 0 维新增 batch 维度
# 假设 txt1 有 5 个 token，txt2 有 5 个 token，则 batch.shape = [2, 5]
batch = torch.stack(batch, dim=0)

# =============================
# 6️ 打印 batch 张量
# 输出: [batch_size, seq_len] 的整数张量
print(batch)


tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [5]:
# ============================================================
# 前向传播演示：使用 DummyGPTModel 对 batch 数据进行推理
# ============================================================

# ----------------------------
# 1️ 设置随机种子
# torch.manual_seed 确保每次初始化权重相同，可复现
torch.manual_seed(123)

# ----------------------------
# 2️ 初始化模型
# DummyGPTModel 仅用于演示，不包含真实 Transformer 层
# 使用 GPT_CONFIG_124M 配置：
# - vocab_size: 词表大小
# - context_length: 序列长度
# - emb_dim: embedding 维度
# - n_layers: transformer 层数
# - drop_rate: embedding dropout
model = DummyGPTModel(GPT_CONFIG_124M)

# ----------------------------
# 3️ 前向传播
# batch 是形状 [batch_size, seq_len] 的整数张量
# logits 是形状 [batch_size, seq_len, vocab_size] 的张量
# 表示每个 token 在词表上对应的预测分数（未 softmax）
logits = model(batch)

# ----------------------------
# 4️ 打印输出形状
# shape = [batch_size, seq_len, vocab_size]
print("Output shape:", logits.shape)

# ----------------------------
# 5️ 打印实际 logits 值
# 对 Dummy 模型来说，输出只是 embedding + 线性映射，没有真正的 Transformer 运算
print(logits)


Output shape: torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6754, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)


---

**注意**

- 如果你在 **Windows** 或 **Linux** 系统上运行这段代码，输出结果可能如下所示：
    

    
```
Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)
```


- 由于这些只是随机数，因此无需担心，可以继续学习本章的后续内容。  
- 造成这种差异的一个可能原因是 `nn.Dropout` 在不同操作系统上的行为不同，这取决于 PyTorch 的编译方式。详细讨论可参考 [PyTorch 问题追踪页面](https://github.com/pytorch/pytorch/issues/121595)。

---

## 4.2 使用层归一化对激活值进行标准化


- 层归一化（Layer Normalization，简称 LayerNorm，[Ba 等人, 2016](https://arxiv.org/abs/1607.06450)）是一种将神经网络层的激活值中心化到均值为 0，并将其方差归一化为 1 的方法。  
- 这种方法可以稳定训练过程，并加速模型权重的有效收敛。  
- 层归一化在 Transformer 模块中被应用于多头注意力机制模块的前后（我们将在后续实现），同时也在最终输出层之前被使用。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/05.webp" width="400px">

- 我们来看一下层归一化（layer normalization）是如何工作的，通过将一个小的输入样本传入一个简单的神经网络层：


In [6]:
# ============================================================
# PyTorch 前向传播示例：Sequential + Linear + ReLU
# ============================================================

# =============================
# 1️ 设置随机种子
# torch.manual_seed 确保权重初始化可复现
torch.manual_seed(123)

# =============================
# 2️ 创建一个 batch 示例
# batch_example 形状: [2, 5]
# - 2 个样本 (batch size = 2)
# - 每个样本有 5 个特征 (feature_dim = 5)
batch_example = torch.randn(2, 5) 

# =============================
# 3️ 定义一个简单的前向传播层
# nn.Sequential 可以把多个层顺序组合
# 这里包含两个层:
# - nn.Linear(5, 6): 输入维度 5，输出维度 6
# - nn.ReLU(): 激活函数，将负值置为 0，正值保持不变
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())

# =============================
# 4️ 前向传播
# 输入 batch_example，通过 Sequential 计算输出
# out 形状: [2, 6]，与 Linear 层输出维度对应
out = layer(batch_example)

# =============================
# 5️ 打印结果
print(out)


tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


- 我们来计算上面2个输入样本的均值和方差：


In [7]:
# ============================================================
# 计算每个样本的均值和方差（按特征维度）
# ============================================================

# =============================
# 1️ mean: 计算每个样本的特征均值
# 参数说明：
# - dim=-1: 在最后一个维度（特征维度）上计算均值
# - keepdim=True: 保持原来的维度，便于后续广播运算
mean = out.mean(dim=-1, keepdim=True)

# =============================
# 2️ var: 计算每个样本的特征方差
# 参数说明：
# - dim=-1: 在最后一个维度（特征维度）上计算方差
# - keepdim=True: 保持维度一致，便于广播
var = out.var(dim=-1, keepdim=True)

# =============================
# 3️ 打印结果
print("Mean:\n", mean)       # 每个样本的均值，形状: [batch_size, 1]
print("Variance:\n", var)    # 每个样本的方差，形状: [batch_size, 1]


Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


- 归一化会独立地应用于两个输入（行）；使用 dim=-1 表示在最后一个维度上进行计算（在此例中为特征维度），而不是在行维度上进行。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/06.webp" width="400px">

- 通过减去均值并除以方差的平方根（标准差），可以使输入在列（特征）维度上中心化，均值为0，方差为1：


In [8]:
# ============================================================
# 对 layer 输出进行手动归一化（类似 LayerNorm 的操作）
# ============================================================

# =============================
# 1️ 标准化（均值为 0，方差为 1）
# out_norm = (out - mean) / sqrt(var)
# 参数解释：
# - out: 原始层输出 [batch_size, features]
# - mean: 每个样本的特征均值 [batch_size, 1]
# - var: 每个样本的特征方差 [batch_size, 1]
# - torch.sqrt(var): 计算标准差
out_norm = (out - mean) / torch.sqrt(var)

# =============================
# 2️ 打印标准化后的输出
print("Normalized layer outputs:\n", out_norm)
# 经过标准化，每个样本的特征均值接近 0，方差接近 1

# =============================
# 3️ 验证标准化效果
# 再次计算标准化后的均值和方差
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)

# =============================
# 4️ 打印结果
print("Mean:\n", mean)       # 每个样本归一化后的均值
print("Variance:\n", var)    # 每个样本归一化后的方差


Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[9.9341e-09],
        [0.0000e+00]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


- 每个输入都以0为中心，方差为1；为了提高可读性，我们可以禁用 PyTorch 的科学计数法：


In [9]:
# ============================================================
# 设置打印选项，禁止科学计数法显示
# ============================================================

# ----------------------------
# torch.set_printoptions(sci_mode=False)
# 参数解释：
# - sci_mode=False ：禁止使用科学计数法（例如 1.23e-2）
#   输出会以普通小数形式显示，更便于阅读
torch.set_printoptions(sci_mode=False)

# ----------------------------
# 打印归一化后每个样本的均值和方差
print("Mean:\n", mean)       # 每个样本归一化后的均值
print("Variance:\n", var)    # 每个样本归一化后的方差


Mean:
 tensor([[0.0000],
        [0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


- 上面，我们对每个输入的特征进行了归一化  
- 现在，使用相同的思路，我们可以实现一个 `LayerNorm` 类：


In [10]:
# ============================================================
# 自定义 LayerNorm 层
# ============================================================

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        """
        自定义 LayerNorm 类，用于对每个样本的最后一维进行归一化。

        参数：
        - emb_dim: embedding 的维度，也就是最后一维的大小
        """
        super().__init__()
        self.eps = 1e-5  # 防止除零的极小值
        # 可学习缩放参数 gamma，初始化为 1
        self.scale = nn.Parameter(torch.ones(emb_dim))
        # 可学习平移参数 beta，初始化为 0
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        """
        前向传播：
        对输入 x 的最后一维进行归一化

        参数：
        - x: 输入张量，形状为 [batch_size, ..., emb_dim]

        返回：
        - 归一化后的张量，形状同输入
        """
        # 计算每个样本最后一维的均值，keepdim=True 保持维度以便广播
        mean = x.mean(dim=-1, keepdim=True)

        # 计算每个样本最后一维的方差，unbiased=False 使用无偏估计 False
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        # 标准化公式：(x - mean) / sqrt(var + eps)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)

        # 应用可学习的缩放和平移参数
        return self.scale * norm_x + self.shift


**缩放与偏移**

- 注意，除了通过减去均值并除以方差来进行归一化之外，我们还添加了两个可训练参数：`scale` 和 `shift`
- 初始的 `scale`（乘以1）和 `shift`（加0）值不会产生任何影响；但是，`scale` 和 `shift` 是可训练参数，LLM 会在训练过程中根据任务性能自动调整它们
- 这允许模型学习适合其处理数据的最佳缩放和偏移
- 注意，在计算方差平方根之前，我们还添加了一个小值（`eps`），以避免方差为0时的除零错误

**有偏方差**
- 在上述方差计算中，设置 `unbiased=False` 表示使用公式 $\frac{\sum_i (x_i - \bar{x})^2}{n}$ 来计算方差，其中 n 是样本数量（这里是特征或列的数量）；该公式不包含 Bessel 校正（即分母为 `n-1`），因此得到的是方差的有偏估计
- 对于嵌入维度 `n` 很大的 LLM 来说，使用 n 与 `n-1` 的差异可以忽略不计
- 但是，GPT-2 在归一化层中使用了有偏方差，因此我们为了与后续章节加载的预训练权重兼容，也采用了此设置

- 现在我们来实际尝试 `LayerNorm`：


In [11]:
# ============================================================
# 使用自定义 LayerNorm 层进行归一化
# ============================================================

# 假设 batch_example 是之前创建的示例张量
# batch_example.shape = [2, 5]，2 个样本，每个样本 5 维特征
# print(batch_example) 可查看原始值

# 初始化 LayerNorm，指定 embedding 维度为 5
ln = LayerNorm(emb_dim=5)

# 前向传播，将 batch_example 输入到 LayerNorm 中
# out_ln.shape = [2, 5]，与输入形状相同
out_ln = ln(batch_example)

# 打印归一化后的输出
print("LayerNorm 输出：\n", out_ln)


LayerNorm 输出：
 tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)


In [12]:
# ============================================================
# 验证 LayerNorm 输出的均值和方差
# ============================================================

# 对每个样本（按行）计算均值
# dim=-1 表示对最后一维求均值，即每个样本的 5 个特征
# keepdim=True 保持维度不变，输出 shape = [2, 1]
mean = out_ln.mean(dim=-1, keepdim=True)

# 对每个样本（按行）计算方差
# dim=-1 表示对最后一维求方差，即每个样本的 5 个特征
# unbiased=False 使用无偏估计（N 而不是 N-1）
# keepdim=True 保持维度不变，输出 shape = [2, 1]
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)

# 打印每个样本归一化后的均值和方差
print("Mean:\n", mean)
print("Variance:\n", var)


Mean:
 tensor([[-0.0000],
        [ 0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/07.webp" width="400px">

## 4.3 使用 GELU 激活函数实现前馈神经网络


在本节中，我们实现一个小型神经网络子模块，它作为大语言模型（LLMs）中 Transformer 块的一部分使用

我们从激活函数开始

在深度学习中，ReLU（Rectified Linear Unit，修正线性单元）激活函数由于其简单性和在各种神经网络架构中的有效性而被广泛使用

在大语言模型中，除了传统的 ReLU，还使用了多种其他类型的激活函数；两个显著的例子是 GELU（Gaussian Error Linear Unit，高斯误差线性单元）和 SwiGLU（Swish-Gated Linear Unit，Swish 门控线性单元）

GELU 和 SwiGLU 是更复杂、平滑的激活函数，分别结合了高斯函数和 Sigmoid 门控线性单元，相比于 ReLU 简单的分段线性函数，它们能为深度学习模型提供更好的性能

- GELU（[Hendrycks 和 Gimpel 2016](https://arxiv.org/abs/1606.08415)）可以通过多种方式实现；其精确定义为 GELU(x) = x ⋅ Φ(x)，其中 Φ(x) 是标准高斯分布的累积分布函数。
- 在实际应用中，通常会使用一种计算上更便宜的近似实现：  
  $$\text{GELU}(x) \approx 0.5 \cdot x \cdot \left(1 + \tanh\left[\sqrt{\frac{2}{\pi}} \cdot \left(x + 0.044715 \cdot x^3\right)\right]\right)$$  
  （原始 GPT-2 模型也是用这个近似进行训练的）


In [13]:
# ============================================================
# GELU 激活函数模块
# ============================================================

class GELU(nn.Module):
    """
    GELU (Gaussian Error Linear Unit) 激活函数模块实现
    - GELU 是 Transformer / GPT 系列模型常用的激活函数
    - 数学公式近似：GELU(x) ≈ 0.5 * x * (1 + tanh(√(2/π) * (x + 0.044715 * x^3)))
    """

    def __init__(self):
        super().__init__()
        # 无需额外参数，GELU 完全由输入计算

    def forward(self, x):
        """
        前向计算

        参数：
        - x: 输入张量，任意形状

        返回：
        - GELU 激活后的张量，形状与输入一致
        """
        # -------------------------------------------------------
        # 公式说明：
        # 1) torch.pow(x, 3) -> x^3
        # 2) x + 0.044715 * x^3 -> 输入加上非线性项
        # 3) torch.sqrt(2/π) * (...) -> 缩放因子
        # 4) torch.tanh(...) -> 非线性映射到 [-1,1]
        # 5) 1 + tanh(...) -> 范围 [0,2]
        # 6) 0.5 * x * (...) -> 最终输出，保留输入 x 的符号和幅度信息
        # -------------------------------------------------------
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))


In [ ]:
# ============================================================
# 可视化 GELU 与 ReLU 激活函数
# ============================================================

import matplotlib.pyplot as plt

# -----------------------------------------------------------
# 初始化激活函数
# -----------------------------------------------------------
gelu, relu = GELU(), nn.ReLU()

# -----------------------------------------------------------
# 创建示例输入数据
# torch.linspace(-3, 3, 100) -> 在 [-3, 3] 之间生成 100 个等间隔点
# -----------------------------------------------------------
x = torch.linspace(-3, 3, 100)

# -----------------------------------------------------------
# 计算激活函数输出
# y_gelu: GELU 输出
# y_relu: ReLU 输出
# -----------------------------------------------------------
y_gelu, y_relu = gelu(x), relu(x)

# -----------------------------------------------------------
# 创建图形，设置大小为 8x3 英寸
# -----------------------------------------------------------
plt.figure(figsize=(8, 3))

# -----------------------------------------------------------
# 循环绘制两种激活函数
# enumerate(zip(...), 1) -> i 从 1 开始
# -----------------------------------------------------------
for i, (y, label) in enumerate(zip([y_gelu, y_relu], ["GELU", "ReLU"]), 1):
    plt.subplot(1, 2, i)  # 1 行 2 列，第 i 个子图
    plt.plot(x, y)         # 绘制激活函数曲线
    plt.title(f"{label} activation function") # 设置标题
    plt.xlabel("x")        # x 轴标签
    plt.ylabel(f"{label}(x)") # y 轴标签
    plt.grid(True)         # 显示网格

plt.tight_layout() # 调整子图布局，避免重叠
plt.show()         # 显示图像


- 正如我们所看到的，ReLU 是一个分段线性函数，当输入为正时直接输出输入值；否则输出为零。  
- GELU 是一个平滑的非线性函数，它近似于 ReLU，但对于负值仍有非零梯度（除了大约在 -0.75 附近）。

- 接下来，我们来实现一个小型神经网络模块 `FeedForward`，该模块将在后续 LLM 的 Transformer 块中使用：


In [14]:
# ============================================================
# 定义 Transformer 中的前馈网络（FeedForward）
# ============================================================

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # -------------------------------------------------------
        # 使用 nn.Sequential 构建顺序网络
        # -------------------------------------------------------
        self.layers = nn.Sequential(
            # 线性层：输入维度 emb_dim -> 输出维度 4*emb_dim
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),

            # GELU 激活函数：非线性变换
            GELU(),

            # 线性层：将维度从 4*emb_dim 投影回 emb_dim
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        # -------------------------------------------------------
        # 前向传播：依次通过线性层、GELU 激活、线性层
        # 输入 x: [batch_size, seq_len, emb_dim]
        # 输出: 与输入维度相同 [batch_size, seq_len, emb_dim]
        # -------------------------------------------------------
        return self.layers(x)


In [15]:
print(GPT_CONFIG_124M["emb_dim"])

768


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/09.webp?12" width="400px">

In [16]:
# ============================================================
# 测试 FeedForward 层
# ============================================================

# 初始化 FeedForward 模块
ffn = FeedForward(GPT_CONFIG_124M)

# 构造示例输入：
# batch_size=2, num_tokens=3, emb_dim=768
x = torch.rand(2, 3, 768)  # 随机生成张量作为输入

# 前向传播
out = ffn(x)

# 输出结果维度
print("Output shape:", out.shape)


Output shape: torch.Size([2, 3, 768])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/10.webp" width="400px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/11.webp" width="400px">

## 4.4 添加捷径连接（Shortcut Connections）


- 接下来，我们来讨论 shortcut 连接的概念，也称为跳跃连接（skip connection）或残差连接（residual connection）。  
- 最初，shortcut 连接是在计算机视觉的深度网络（残差网络，ResNet）中提出的，用于缓解梯度消失问题。  
- shortcut 连接为梯度在网络中流动提供了一条更短的替代路径。  
- 这是通过将某一层的输出加到后面某一层的输出上实现的，通常会跳过中间的一层或多层。  
- 让我们用一个小型示例网络来说明这个想法：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/12.webp?123" width="400px">


- 在代码中，它的实现如下：


In [20]:
# ============================================================
# 示例深度神经网络 (带可选残差连接)
# ============================================================

class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        """
        初始化深度神经网络
        
        参数：
        - layer_sizes: list，表示每一层的输入输出维度
          例如 [10, 20, 30, 40, 50, 60] 表示：
          第1层 10->20，第2层 20->30，第3层 30->40，第4层 40->50，第5层 50->60
        - use_shortcut: bool，是否使用残差连接
        """
        super().__init__()
        self.use_shortcut = use_shortcut

        # 创建 5 层全连接网络，每层后面接 GELU 激活
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

    def forward(self, x):
        """
        前向传播
        - 对每一层：
          1. 计算当前层输出 layer_output
          2. 如果允许残差连接且输入和输出形状相同，则执行 x + layer_output
          3. 否则直接更新 x = layer_output

            条件说明：
            
            1. self.use_shortcut
               - 类型：bool
               - 含义：是否启用残差连接（shortcut）
               - 如果为 True，则允许在前向传播中将输入 x 与当前层输出 layer_output 相加
            
            2. x.shape == layer_output.shape
               - 类型：tuple 比较
               - 含义：判断输入 x 的形状是否与当前层输出 layer_output 的形状一致
               - 残差连接要求输入与输出的维度必须相同，否则无法直接相加
                 例如，(batch_size, 20) 与 (batch_size, 20) 可以相加
                       (batch_size, 20) 与 (batch_size, 30) 不可以相加
            
            总结：
            - 只有当“允许使用残差连接”且“输入输出维度匹配”时，才执行 x + layer_output
            - 否则，不使用残差，直接用 layer_output 作为新的 x
        """
        for layer in self.layers:
            layer_output = layer(x)  # 当前层输出
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output  # 残差连接
            else:
                x = layer_output       # 普通前向传播
        return x


def print_gradients(model, x):
    """
    计算模型梯度并打印每层权重的平均绝对梯度
    
    参数：
    - model: 待测试的 nn.Module 模型
    - x: 输入张量
    """
    # 前向传播
    output = model(x)
    
    # 构造一个简单目标
    target = torch.tensor([[0.]])

    # 定义均方误差损失
    loss_fn = nn.MSELoss()
    loss = loss_fn(output, target)
    
    # 反向传播，计算梯度
    loss.backward()

    # 遍历模型参数，打印每个权重的平均绝对梯度
    # 遍历模型的所有参数（权重和偏置）
    for name, param in model.named_parameters():
        
        # 判断当前参数是否是权重（weight），排除偏置（bias）
        if 'weight' in name:
            
            # param.grad 是该权重的梯度张量
            # param.grad.abs() 取梯度的绝对值
            # param.grad.abs().mean() 计算所有元素的平均值
            # .item() 将单元素张量转换为 Python 数值
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")


- 我们先打印 **未使用** 捷径连接时的梯度值：


In [21]:
# -----------------------------
# 定义神经网络的层尺寸
layer_sizes = [3, 3, 3, 3, 3, 1]
# 含义：
# - 输入维度：3
# - 5 层隐藏层，每层维度分别为 3（注意此例中所有隐藏层维度相同）
# - 输出维度：1
# -----------------------------

sample_input = torch.tensor([[1., 0., -1.]])  
# 创建一个单样本输入向量，维度为 [1, 3]
# 数据示例：[1, 0, -1]

torch.manual_seed(123)  
# 设置随机种子，使后续权重初始化可复现

# -----------------------------
# 创建不使用残差连接的神经网络
# use_shortcut=False 表示不启用残差连接
# -----------------------------
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=False
)

# -----------------------------
# 计算并打印各层权重的梯度
# 1. 先前向传播得到输出
# 2. 使用 MSELoss 与目标 [[0.]] 计算损失
# 3. 反向传播计算梯度
# 4. 遍历模型参数，打印每层权重的平均绝对梯度
# -----------------------------
print_gradients(model_without_shortcut, sample_input)


layers.0.0.weight has gradient mean of 0.00020173587836325169
layers.1.0.weight has gradient mean of 0.0001201116101583466
layers.2.0.weight has gradient mean of 0.0007152041653171182
layers.3.0.weight has gradient mean of 0.001398873864673078
layers.4.0.weight has gradient mean of 0.005049646366387606


- Next, let's print the gradient values **with** shortcut connections:

In [22]:
# 设置随机种子，保证每次生成的权重一致
torch.manual_seed(123)

# 创建一个使用残差连接（shortcut）的深度神经网络实例
model_with_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes,      # 层的大小列表 [3, 3, 3, 3, 3, 1]
    use_shortcut=True # 开启残差连接
)

# 打印每层权重参数的平均梯度
print_gradients(model_with_shortcut, sample_input)


layers.0.0.weight has gradient mean of 0.22169792652130127
layers.1.0.weight has gradient mean of 0.20694106817245483
layers.2.0.weight has gradient mean of 0.32896995544433594
layers.3.0.weight has gradient mean of 0.2665732502937317
layers.4.0.weight has gradient mean of 1.3258541822433472


- 从上面的输出可以看出，捷径连接可以防止早期层（如 `layer.0`）的梯度消失  
- 在接下来实现 Transformer 块时，我们将使用这一捷径连接的概念


## 4.5 在 Transformer 块中连接注意力层和线性层


- 在本节中，我们将之前的概念结合起来，形成所谓的 Transformer 块。  
- 一个 Transformer 块将上一章介绍的因果多头注意力（causal multi-head attention）模块与线性层以及我们在前面章节实现的前馈神经网络（feed forward neural network）结合起来。  
- 此外，Transformer 块还使用了 dropout 和 shortcut 连接。


In [24]:
# -------------------------------------------------------
# TransformerBlock 类：单个 Transformer 层（带残差连接和 LayerNorm）
# -------------------------------------------------------

# 如果本地没有 `previous_chapters.py`，可以从 `llms-from-scratch` PyPI 包导入
# 示例：
# from llms_from_scratch.ch03 import MultiHeadAttention

# -------------------------------------------------------
# 包含：
#   1) 多头自注意力子层（Multi-Head Self-Attention）
#   2) 前馈全连接子层（FeedForward）
#   3) 残差连接（Residual / Shortcut Connection）
#   4) 层归一化（LayerNorm）
#   5) Dropout 正则化
# 输入输出 shape： [batch_size, num_tokens, emb_dim]
# -------------------------------------------------------

from previous_chapters import MultiHeadAttention
import torch
import torch.nn as nn

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        # ---------------------------
        # 1. 多头自注意力层
        # ---------------------------
        # d_in: 输入嵌入维度
        # d_out: 输出嵌入维度（通常等于输入嵌入维度）
        # context_length: 最大上下文长度（每个序列的 token 数量上限）
        # num_heads: 注意力头数量
        # dropout: 注意力输出后的 dropout 概率
        # qkv_bias: 是否在 query/key/value 矩阵中添加偏置
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        
        # ---------------------------
        # 2. 前馈全连接层（FeedForward）
        # ---------------------------
        # 通常包含两层线性 + GELU 激活
        # 扩展维度到 4*emb_dim 后再降回 emb_dim
        self.ff = FeedForward(cfg)
        
        # ---------------------------
        # 3. 层归一化（LayerNorm）
        # ---------------------------
        # norm1 用于注意力子层
        # norm2 用于前馈子层
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        
        # ---------------------------
        # 4. 残差连接后的 Dropout
        # ---------------------------
        # 用于防止过拟合
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # ===========================
        # 注意力子层 forward
        # ===========================
        
        # 1) 保存残差输入
        shortcut = x  # shape: [batch_size, num_tokens, emb_dim]
        
        # 2) 对输入做 LayerNorm
        x = self.norm1(x)
        
        # 3) 通过多头自注意力层
        x = self.att(x)  # shape: [batch_size, num_tokens, emb_dim]
        
        # 4) 残差 dropout
        x = self.drop_shortcut(x)
        
        # 5) 添加残差连接
        x = x + shortcut
        
        # ===========================
        # 前馈子层 forward
        # ===========================
        
        # 1) 保存残差输入
        shortcut = x  # shape: [batch_size, num_tokens, emb_dim]
        
        # 2) LayerNorm
        x = self.norm2(x)
        
        # 3) 前馈网络
        x = self.ff(x)  # shape: [batch_size, num_tokens, emb_dim]
        
        # 4) 残差 dropout
        x = self.drop_shortcut(x)
        
        # 5) 添加残差连接
        x = x + shortcut

        # 返回最终输出
        # 输出 shape: [batch_size, num_tokens, emb_dim]
        return x


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/13.webp?1" width="400px">

- 假设我们有 2 个输入样本，每个样本包含 6 个 token，每个 token 是一个 768 维的嵌入向量；那么这个 Transformer 块会先应用自注意力（self-attention），然后经过线性层，生成一个大小相似的输出。  
- 你可以将输出理解为上一章讨论的上下文向量（context vectors）的增强版本。


In [25]:
# 设置随机种子，保证每次运行生成的随机数一致，便于复现结果
torch.manual_seed(123)

# ===========================
# 准备输入张量 x
# ===========================
# batch_size = 2：有 2 个样本
# num_tokens = 4：每个样本包含 4 个 token
# emb_dim = 768：每个 token 的 embedding 维度为 768
x = torch.rand(2, 4, 768)  # 随机生成输入张量

# ===========================
# 初始化 TransformerBlock
# ===========================
# 使用之前定义的 DummyGPT 配置
block = TransformerBlock(GPT_CONFIG_124M)

# ===========================
# 前向传播
# ===========================
# 通过 TransformerBlock 对输入 x 进行处理
# 包含：
#   1) 多头自注意力
#   2) 残差连接 + LayerNorm + Dropout
#   3) 前馈网络
#   4) 残差连接 + LayerNorm + Dropout
output = block(x)

# ===========================
# 打印输入和输出的形状
# ===========================
# 输入 shape: [batch_size, num_tokens, emb_dim] = [2, 4, 768]
# 输出 shape 与输入保持一致，因为 TransformerBlock 是残差结构，embedding 维度不变
print("Input shape:", x.shape)
print("Output shape:", output.shape)


Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/14.webp?1" width="400px">

## 4.6 编写 GPT 模型代码


- 我们几乎完成了：现在让我们将 Transformer 块插入到本章开头编写的架构中，从而得到一个可用的 GPT 架构。  
- 注意，Transformer 块会被重复多次；以最小的 124M GPT-2 模型为例，我们会重复 12 次：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/15.webp" width="400px">

- 对应的代码实现，其中 `cfg["n_layers"] = 12`：


In [26]:
# ===============================================
# 定义 GPTModel 类
# ===============================================
# 这是一个简化的 GPT 模型，包含：
# 1) Token embedding
# 2) Position embedding
# 3) 多个 TransformerBlock 堆叠
# 4) LayerNorm
# 5) 输出投影到词汇表大小
# ===============================================

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        # -------------------------------
        # Token embedding
        # 将输入的 token 索引映射为 embedding 向量
        # cfg["vocab_size"]: 词表大小
        # cfg["emb_dim"]: embedding 维度
        # 输出 shape: [batch_size, seq_len, emb_dim]
        # -------------------------------
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        
        # -------------------------------
        # Position embedding
        # 为序列中的每个位置生成固定长度的向量
        # cfg["context_length"]: 序列最大长度
        # 输出 shape: [seq_len, emb_dim]
        # -------------------------------
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        
        # -------------------------------
        # Dropout 层，用于 embedding 层
        # cfg["drop_rate"]: dropout 概率
        # -------------------------------
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # -------------------------------
        # TransformerBlock 堆叠
        # cfg["n_layers"]: TransformerBlock 层数
        # 使用 nn.Sequential 将多个 TransformerBlock 组合
        # 每个 TransformerBlock 内部包含：
        #   - 多头自注意力
        #   - 前馈网络
        #   - LayerNorm
        #   - 残差连接
        # 输出 shape: [batch_size, seq_len, emb_dim]
        # -------------------------------
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        
        # -------------------------------
        # 最后的 LayerNorm 层
        # 用于归一化最后 Transformer 输出
        # -------------------------------
        self.final_norm = LayerNorm(cfg["emb_dim"])
        
        # -------------------------------
        # 输出投影层，将 embedding 映射到词表维度
        # 输出 shape: [batch_size, seq_len, vocab_size]
        # -------------------------------
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        # in_idx: 输入 token 索引张量
        # shape: [batch_size, seq_len]
        batch_size, seq_len = in_idx.shape
        
        # -------------------------------
        # 获取 token embedding
        # shape: [batch_size, seq_len, emb_dim]
        # -------------------------------
        tok_embeds = self.tok_emb(in_idx)
        
        # -------------------------------
        # 获取 position embedding
        # shape: [seq_len, emb_dim]
        # 使用与输入相同的设备
        # -------------------------------
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        
        # -------------------------------
        # 将 token embedding 和 position embedding 相加
        # shape: [batch_size, seq_len, emb_dim]
        # -------------------------------
        x = tok_embeds + pos_embeds
        
        # -------------------------------
        # 对 embedding 应用 dropout
        # -------------------------------
        x = self.drop_emb(x)
        
        # -------------------------------
        # 通过堆叠的 TransformerBlock
        # 每个 block 保留输入输出维度一致
        # shape: [batch_size, seq_len, emb_dim]
        # -------------------------------
        x = self.trf_blocks(x)
        
        # -------------------------------
        # 对最后输出做 LayerNorm
        # -------------------------------
        x = self.final_norm(x)
        
        # -------------------------------
        # 投影到词表维度，得到每个 token 对每个词的 logits
        # shape: [batch_size, seq_len, vocab_size]
        # -------------------------------
        logits = self.out_head(x)
        
        return logits


- 使用 1.24 亿参数模型的配置，我们现在可以用随机初始化权重实例化该 GPT 模型，如下所示：


In [27]:
# ===============================================
# 测试 GPTModel 前向传播
# ===============================================

torch.manual_seed(123)  # 固定随机种子，保证可复现

# 初始化 GPTModel，使用之前定义的配置
model = GPTModel(GPT_CONFIG_124M)

# 将之前准备好的 batch 输入模型
# batch shape: [batch_size, seq_len]，每个元素是 token ID
out = model(batch)

# 打印输入 batch
print("Input batch (token IDs):\n", batch)

# 打印输出 shape
# 输出 shape: [batch_size, seq_len, vocab_size]
# 对应每个 token 对整个词表的 logits
print("\nOutput shape:", out.shape)

# 打印实际输出 logits
# 注意，这里的 logits 是未经过 softmax 的原始分数
print("\nOutput logits:\n", out)


Input batch (token IDs):
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])

Output logits:
 tensor([[[ 0.3613,  0.4222, -0.0711,  ...,  0.3483,  0.4661, -0.2838],
         [-0.1792, -0.5660, -0.9485,  ...,  0.0477,  0.5181, -0.3168],
         [ 0.7120,  0.0332,  0.1085,  ...,  0.1018, -0.4327, -0.2553],
         [-1.0076,  0.3418, -0.1190,  ...,  0.7195,  0.4023,  0.0532]],

        [[-0.2564,  0.0900,  0.0335,  ...,  0.2659,  0.4454, -0.6806],
         [ 0.1230,  0.3653, -0.2074,  ...,  0.7705,  0.2710,  0.2246],
         [ 1.0558,  1.0318, -0.2800,  ...,  0.6936,  0.3205, -0.3178],
         [-0.1565,  0.3926,  0.3288,  ...,  1.2630, -0.1858,  0.0388]]],
       grad_fn=<UnsafeViewBackward0>)


- 我们将在下一章训练该模型  
- 不过，关于模型大小有一点需要说明：之前我们称其为 1.24 亿参数模型；我们可以通过以下方法再次确认这个数字：


In [28]:
# ===============================================
# 统计 GPT 模型总参数量
# ===============================================

# 遍历模型的所有参数 tensor，p.numel() 返回该 tensor 中元素总数
total_params = sum(p.numel() for p in model.parameters())

# 打印总参数量，使用千位分隔符提高可读性
print(f"Total number of parameters: {total_params:,}")


Total number of parameters: 163,009,536


- 如上所示，这个模型有 1.63 亿参数，而不是 1.24 亿参数；这是为什么呢？  
- 在原始 GPT-2 论文中，研究人员使用了权重共享（weight tying），也就是说，他们重复使用了 token 嵌入层（`tok_emb`）作为输出层，即设置 `self.out_head.weight = self.tok_emb.weight`。  
- token 嵌入层将 50,257 维的 one-hot 编码输入 token 映射到 768 维的嵌入表示。  
- 输出层将 768 维的嵌入映射回 50,257 维的表示，以便我们可以将这些表示转换回单词（关于这部分内容将在下一节详细讲解）。  
- 因此，嵌入层和输出层具有相同数量的权重参数，这可以通过它们权重矩阵的形状看出来。  
- 但是，有关模型大小的一个小说明：我们之前称其为 1.24 亿参数模型；我们可以通过以下方法再次验证这个数字：


In [29]:
# ===============================================
# 查看 GPT 模型关键层的权重形状
# ===============================================

# 打印 token embedding 层权重矩阵的形状
# model.tok_emb.weight.shape -> [vocab_size, emb_dim]
print("Token embedding layer shape:", model.tok_emb.weight.shape)

# 打印输出线性层（logits projection）的权重矩阵形状
# model.out_head.weight.shape -> [vocab_size, emb_dim]
print("Output layer shape:", model.out_head.weight.shape)


Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


- 在原始 GPT-2 论文中，研究人员重复使用了 token 嵌入矩阵作为输出矩阵。  
- 相应地，如果我们减去输出层的参数数量，就会得到一个 1.24 亿参数的模型：


In [30]:
# ===============================================
# 计算考虑 weight tying 后的 GPT 模型训练参数总数
# ===============================================

# total_params 已经包含了整个模型的参数总数
# 由于输出层 out_head 与 token embedding 层共享权重（weight tying），
# 所以 out_head 的参数不再单独计算
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())

print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")


Number of trainable parameters considering weight tying: 124,412,160


- 在实际操作中，我发现不使用权重共享（weight-tying）训练模型更为简单，这也是我们在这里没有实现它的原因。  
- 不过，当我们在第 5 章加载预训练权重时，会重新考虑并应用这一权重共享的想法。  
- 最后，我们可以如下计算模型的内存需求，这可以作为一个有用的参考：


In [31]:
# ===============================================
# 计算 GPT 模型参数占用的内存大小
# ===============================================

# 假设模型所有参数均为 float32 类型，每个参数占用 4 字节
total_size_bytes = total_params * 4

# 将字节转换为 MB
total_size_mb = total_size_bytes / (1024 * 1024)

print(f"Total size of the model: {total_size_mb:.2f} MB")


Total size of the model: 621.83 MB


- 练习：你也可以尝试以下其他配置，这些配置在 [GPT-2 论文](https://scholar.google.com/citations?view_op=view_citation&hl=en&user=dOad5HoAAAAJ&citation_for_view=dOad5HoAAAAJ:YsMSGLbcyi4C) 中有所提及。

    - **GPT2-small**（我们已经实现的 1.24 亿参数配置）：
        - "emb_dim" = 768
        - "n_layers" = 12
        - "n_heads" = 12

    - **GPT2-medium**：
        - "emb_dim" = 1024
        - "n_layers" = 24
        - "n_heads" = 16
    
    - **GPT2-large**：
        - "emb_dim" = 1280
        - "n_layers" = 36
        - "n_heads" = 20
    
    - **GPT2-XL**：
        - "emb_dim" = 1600
        - "n_layers" = 48
        - "n_heads" = 25


## 4.7 文本生成


- 类似于我们上面实现的 GPT 模型的大型语言模型（LLM）用于一次生成一个单词


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/16.webp" width="400px">

- 下面的 `generate_text_simple` 函数实现了贪婪解码（greedy decoding），这是一种简单且快速的文本生成方法。  
- 在贪婪解码中，每一步模型都会选择概率最高的单词（或 token）作为下一个输出（最高的 logit 对应最高概率，因此从技术上讲，我们甚至不必显式计算 softmax 函数）。  
- 在下一章中，我们将实现一个更高级的 `generate_text` 函数。  
- 下图展示了 GPT 模型在给定输入上下文的情况下，如何生成下一个单词 token：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/13.webp?123" width="400px">


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/17.webp" width="600px">

In [33]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    """
    简单文本生成函数（贪心解码）
    
    参数：
    - model: 语言模型对象，接收 token 索引并输出 logits
    - idx: 当前上下文的 token 索引张量，形状为 (batch, n_tokens)
    - max_new_tokens: 要生成的新 token 数量
    - context_size: 模型支持的最大上下文长度

    返回：
    - idx: 生成后的完整 token 序列，包括原始上下文和新生成的 token
    """

    for _ in range(max_new_tokens):
        # ---------------------------------------------------
        # 如果当前上下文长度超过模型最大支持长度，只取最后 context_size 个 token
        # 例如：模型最多支持 5 个 token，而当前上下文有 10 个 token
        # 则只取最后 5 个 token 作为上下文输入
        # -context_size: 切片操作，用于选择 最后 context_size 个 token
        # ---------------------------------------------------
        idx_cond = idx[:, -context_size:]
        
        # ---------------------------------------------------
        # 获取模型预测 logits
        # torch.no_grad() 表示不计算梯度，节省显存
        # logits 形状：(batch, n_tokens_in_context, vocab_size)
        # ---------------------------------------------------
        with torch.no_grad():
            logits = model(idx_cond)
        
        # ---------------------------------------------------
        # 只关注最后一个 token 的预测
        # logits[:, -1, :] 将 (batch, n_tokens, vocab_size) 转为 (batch, vocab_size)
        # -1 表示 最后一个 token，因为我们只关心“当前上下文生成下一个 token 的概率分布”，所以只取最后一步的预测
        #  如果不取 -1，会得到整个序列的 logits，而我们只需要最后一步
        # ---------------------------------------------------
        logits = logits[:, -1, :]
        
        # ---------------------------------------------------
        # 将 logits 转换为概率分布
        # softmax 在最后一维 (vocab_size) 上计算
        # probas 形状：(batch, vocab_size)
        # ---------------------------------------------------
        probas = torch.softmax(logits, dim=-1)
        
        # ---------------------------------------------------
        # 贪心选择：取概率最大的 token 索引
        # idx_next 形状：(batch, 1)
        # keepdim=True 保持维度，以便后续拼接
        # ---------------------------------------------------
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        
        # ---------------------------------------------------
        # 将新生成的 token 拼接到现有序列中
        # idx 形状：(batch, n_tokens+1)
        # ---------------------------------------------------
        idx = torch.cat((idx, idx_next), dim=1)

    return idx


- 上面的 `generate_text_simple` 实现了一个迭代过程，每次生成一个标记（token）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/18.webp" width="600px">


- 我们来准备一个输入示例：


In [34]:
# 定义初始上下文文本
start_context = "Hello, I am"  
# -----------------------------
# start_context: 字符串，表示模型生成文本的起始上下文
# -----------------------------

# 使用 tokenizer 对字符串进行编码，转换为 token 索引序列
encoded = tokenizer.encode(start_context)  
# encoded: List[int]，每个整数表示对应的词汇表索引
print("encoded:", encoded)  
# 输出编码后的 token 列表，例如 [15496, 11, 314, 307]

# 将编码后的 token 列表转换为 PyTorch 张量
encoded_tensor = torch.tensor(encoded).unsqueeze(0)  
# torch.tensor(encoded): 将 list[int] 转为 1D 张量，形状 [n_tokens]
# .unsqueeze(0): 在第 0 维增加一个 batch 维度
# encoded_tensor 形状: [1, n_tokens]，表示 batch_size=1
print("encoded_tensor.shape:", encoded_tensor.shape)  
# 输出张量形状，例如 torch.Size([1, 4])


encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [35]:
# -----------------------------
# 将模型切换为评估模式（evaluation mode），禁用 dropout
# -----------------------------
model.eval()  
# model.eval(): 将模型置于评估模式，注意力层或其他层的 dropout 不会在 forward 过程中生效

# -----------------------------
# 调用简单文本生成函数 generate_text_simple
# -----------------------------
out = generate_text_simple(
    model=model,                   # 模型对象，负责根据上下文生成 logits
    idx=encoded_tensor,            # 当前上下文的 token 索引张量，形状 [1, n_tokens]
    max_new_tokens=6,              # 生成的新 token 数量
    context_size=GPT_CONFIG_124M["context_length"]  # 模型支持的最大上下文长度
)

# 输出生成后的 token 序列（包括原始上下文和新生成的 token）
print("Output:", out)  
# 例如: tensor([[15496, 11, 314, 307, 2061, 389, 345, 0, 0, ...]])

# 输出生成序列的长度（token 数量）
print("Output length:", len(out[0]))  
# out[0]: batch 中第一个样本的 token 序列
# len(out[0]): 序列长度 = 原始上下文长度 + max_new_tokens


Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


- 移除批量维度（batch dimension）并将其转换回文本：


In [36]:
# -----------------------------
# 将生成的 token 序列解码为可读文本
# -----------------------------
decoded_text = tokenizer.decode(
    out.squeeze(0).tolist()  # 将 [1, n_tokens] 张量压缩为 [n_tokens] 列表
)

# 输出解码后的文本
print(decoded_text)
# tokenizer.decode(): 将 token ID 转换回字符串
# out.squeeze(0): 去掉 batch 维度，因为 batch=1
# .tolist(): 将 tensor 转换为 Python 列表，便于 decode


Hello, I am Featureiman Byeswickattribute argue


- 请注意，该模型尚未经过训练；因此上面的输出文本是随机的  
- 我们将在下一章训练该模型


## 总结与要点

- 请参见 [./gpt.py](./gpt.py) 脚本，这是一个独立的脚本，包含我们在此 Jupyter Notebook 中实现的 GPT 模型  
- 练习答案可以在 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 中找到
